## Григорьев Дмитрий, вариант 3
### Импортируем необходимые модули,в т.ч свои


In [1]:
from math import exp
from mft4 import non_equidistant_interpolation_Newton
from pandas import DataFrame
from mft5 import derivate_experiment, get_table

### Дано: $f(x) = e^{2x},\ x=2, h=0.1 $

In [2]:
f = lambda x: exp(2*x)      #функция
df = lambda x: 2*exp(2*x)   #первая производная
d2f = lambda x: 4*exp(2*x)  #вторая производная
h = 0.1                     #шаг
pivot = 2.0                 #наш x 
start = 1.5                 
degree = 10                 #степень интерполяционного многочлена
k = 5                       #номер знака, до которого округляем значения
nodes = [start + k*h for k in range(degree+1)]

### Таблица, построенная по образцу
### Видим, что погрешность в случае формулы для первой производной первого порядка аппроксимации достаточно велика.
### Это связано с порядком аппроксимации и скоростью роста производных функции (согласно формуле для погрешности).  

In [3]:
dataframe1 = get_table(f, nodes, degree, h, df, d2f)
dataframe1

,$x$,$f(x)$,$f'(x)$,"$\widetilde {f'}(x), \ O(h)$","$погр.,\ O(h)$","$\widetilde{\widetilde {f'}}(x), \ O(h^2)$","$погр., \ O(h^2)$",$f''(x)$,"$\widetilde {f''}(x),\ O(h^2)$","$погр., \ O(h^2)$"
,1.5,20.0855,40.1711,44.4699,-4.29886,39.547,0.624024,80.3421,,
,1.6,24.5325,49.0651,54.3157,-5.25064,49.3928,-0.327755,98.1301,98.4577,-0.327537
,1.7,29.9641,59.9282,66.3413,-6.41314,60.3285,-0.400321,119.856,120.256,-0.400054
,1.8,36.5982,73.1965,81.0295,-7.83303,73.6854,-0.488953,146.393,146.882,-0.488628
,1.9,44.7012,89.4024,98.9697,-9.56729,89.9996,-0.597209,178.805,179.402,-0.596811
,2,54.5982,109.196,120.882,-11.6855,109.926,-0.729433,218.393,219.122,-0.728947
,2.1,66.6863,133.373,147.645,-14.2727,134.264,-0.890931,266.745,267.636,-0.890337
,2.2,81.4509,162.902,180.334,-17.4327,163.99,-1.08819,325.803,326.891,-1.08746
,2.3,99.4843,198.969,220.261,-21.2924,200.298,-1.32911,397.937,399.265,-1.32823
,2.4,121.51,243.021,269.027,-26.0066,244.644,-1.62338,486.042,487.664,-1.6223


### Функция, вычисляющая первую производную с четвёртым порядком аппроксимации

In [4]:
def derivative1(f, h, pivot, nodes):           #вычисление производной с четвёртым порядком аппроксимации
    f1 = round(non_equidistant_interpolation_Newton(f, nodes,pivot-2*h, degree),k)
    f2 = round(non_equidistant_interpolation_Newton(f, nodes,pivot-h, degree),k)
    f3 = round(non_equidistant_interpolation_Newton(f, nodes,pivot+h, degree),k)
    f4 = round(non_equidistant_interpolation_Newton(f, nodes,pivot+2*h, degree),k)
    return (f1 - 8 * f2 + 8 * f3 - f4)/(12 * h)

### $\widetilde{f'}(x)=\frac{f(x-2h)-8f(x-h)+8f(x+h)-f(x+2h)}{12h}$ 
#### $\epsilon = 5 \cdot 10^{-6}$
### $|R_{10}^{(1)}(f)|=|f'(x)-\widetilde{f'_p}(x)|\le|f'(x)-\widetilde {f'}(x)|+|\widetilde {f'}(x)-\widetilde {f'_p}(x)|\le \frac{2}{3}h^4 \cdot M_5+\frac{3\epsilon}{2h}$
#### $M_5=\max_\limits{x\in [1.5,2.5]}|f^{(V)}(x)|$
#### $\widetilde{f'_p}$ -- значение производной,посчитанное с помощью приближения функции многочленом
### Вычислим $h$ оптимальное экспериментально (ниже) и теоретически (ищем точку минимума оценки погрешности (выше) с помощью дифференциального исчисления) 

In [5]:
dataframe2 = derivate_experiment(derivative1,f,h,pivot, nodes, df)
dataframe2.columns.name = "$O(h^4)$"
dataframe2

$O(h^4)$,0,1,2,3
$h$,0.100000,0.05000,0.025000,0.0125
Численное значение производной,109.190467,109.19595,109.196267,109.1962
Погрешность,0.005833,0.00035,0.000033,0.0001


#### $h_{exp} = 0.0125,\ h_{th} = \sqrt[5]{\frac{9 \epsilon}{16M_5}}\approx0.01427$ -- оптимальное h практ. и теор. соответсвенно

